In [2]:
#### Consolidate code from Geo Notebooks into a single block now that data is in the right format

### Import dataframe with geoghraphical of baseball fields

import pandas as pd
import geopy
from geopy import Point, distance


file_path = ('data/kml/ballparks.kml')


## Parse the kml file using the BeautifulSoup library
from bs4 import BeautifulSoup

## Initialize the BeautifulSoup object

with open(file_path) as f:
    xml_data = f.read()

soup = BeautifulSoup(xml_data, 'xml')

## Get the coordinates from the kml file
folders = soup.Document.Folder.find_all('Folder')
list = soup.Document.Folder.find_all('Folder')

### Create a dataframe with the coordinates of the baseball fields
df = pd.DataFrame(columns=['name', 'foul', 'fop'])

## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    }

    i+=1

    df = df.append(row, ignore_index=True)



## Clean up the data
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]

### Extract the home_plate coordinates by takeing the first coordinate in the foul coordinates column
df['home_plate'] = df['foul'].str.split(' ').str[0]

## drop the last 2 characters from the home_plate coordinates
df['home_plate'] = df['home_plate'].str[:-2]

### drop every ',0' from the fop coordinates
df['fop'] = df['fop'].str.replace(',0', '')

### reverse the order of the coordinates in the home_plate column
df['home_plate'] = df['home_plate'].str.split(',').str[::-1].str.join(',')

### Creaty a list of tuples for the fop coordinates
df['fop_list'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])

## Reverse the order of the values each coordinate tuple
df['fop_list'] = df['fop_list'].apply(lambda x: [tuple(reversed(coord)) for coord in x])



C:\Users\Justin\AppData\Local\Temp\ipykernel_1236\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_1236\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_1236\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_1236\369921160.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\i

In [3]:

### use the geopy distance method to get the distance between home_plate and each coordinate in the fop_list column for each row of the dataframe


### calculate the distance between home_plate and each coordinate in the fop_list column
df['distance'] = df.apply(lambda x: [distance.distance(x['home_plate'], coord).ft for coord in x['fop_list']], axis=1)

df['distance']

### drop any 0s from the distance column

df['distance'] = df['distance'].apply(lambda x: [i for i in x if i != 0])

## get the min from the distance column

df['min_distance'] = df['distance'].apply(lambda x: min(x))

### get the max distance from home_plate to the fop coordinates
df['max_distance'] = df['distance'].apply(lambda x: max(x))

df.head()

### get the average distance from home_plate to the fop coordinates
df['avg_distance'] = df['distance'].apply(lambda x: sum(x)/len(x))

### output the dataframe to a csv file

df.to_csv('data/viz/distance.csv', index=False)


#### FUCK YES IT WORKS!!!!

### some straNGE values, mostly in the min column. They all seem to be plots i did very early in the process.
### Those fields were probably maped differently than the process I eventually developed. I'll have to go back and fix them.



#### Code below has been modified to read a kml file that has been exported from Google My Maps and not yet been aggrigated in Google Earth


## First block of code takes the standard map stype with full foul and fop polygons


In [10]:


### Import dataframe with geoghraphical of baseball fields

import pandas as pd
import geopy
from geopy import Point, distance


file_path = ('data\kml\lansing_replots.kml')


## Parse the kml file using the BeautifulSoup library
from bs4 import BeautifulSoup

## Initialize the BeautifulSoup object

with open(file_path) as f:
    xml_data = f.read()

soup = BeautifulSoup(xml_data, 'xml')

## Get the coordinates from the kml file
folders = soup.Document.find_all('Folder')
list = soup.Document.find_all('Folder')

### Create a dataframe with the coordinates of the baseball fields
df = pd.DataFrame(columns=['name', 'foul', 'fop'])

## Loop through the folders and extract the data


    
i = 0   

for i in range(len(list)):

    folders = list[i]
    field_name = folders.find('name').text
    foul = folders.find_all('coordinates')[0].text
    fop = folders.find_all('coordinates')[1].text
    

    row = {
        'field': field_name,
        'foul': foul,
        'fop': fop
    
    }

    i+=1

    df = df.append(row, ignore_index=True)



## Clean up the data
# remove new line and and space characters from coordinates
df = df.replace(r'\n','', regex=True) 
df = df.replace(r'\t','', regex=True) 


## Drop any duplicate rows
df = df.drop_duplicates(subset=['field'], keep='first')

## Drop and rows with empty fields
df = df[(df != 0).all(1)]

### Extract the home_plate coordinates by takeing the first coordinate in the foul coordinates column
# df['home_plate'] = df['fop'].str.split(' ').str[0]

## drop the last 2 characters from the home_plate coordinates
# df['home_plate'] = df['home_plate'].str[:-2]

### drop every ',0' from the fop coordinates
df['fop'] = df['fop'].str.replace(',0', '')

### reverse the order of the coordinates in the home_plate column
# df['home_plate'] = df['home_plate'].str.split(',').str[::-1].str.join(',')

### Creaty a list of tuples for the fop coordinates
df['fop_list'] = df['fop'].apply(lambda x: [tuple(map(float, coord.split(','))) for coord in x.split()])

## Reverse the order of the values each coordinate tuple
df['fop_list'] = df['fop_list'].apply(lambda x: [tuple(reversed(coord)) for coord in x])

df['home_plate'] = df['fop_list'].apply(lambda x: x[0])

### use the geopy distance method to get the distance between home_plate and each coordinate in the fop_list column for each row of the dataframe


### calculate the distance between home_plate and each coordinate in the fop_list column
df['distance'] = df.apply(lambda x: [distance.distance(x['home_plate'], coord).ft for coord in x['fop_list']], axis=1)

df['distance']

### drop any 0s from the distance column

df['distance'] = df['distance'].apply(lambda x: [i for i in x if i != 0])

## get the min from the distance column

df['min_distance'] = df['distance'].apply(lambda x: min(x))

### get the max distance from home_plate to the fop coordinates
df['max_distance'] = df['distance'].apply(lambda x: max(x))

df.head()

### get the average distance from home_plate to the fop coordinates
df['avg_distance'] = df['distance'].apply(lambda x: sum(x)/len(x))

### output the dataframe to a csv file

df.to_csv('TEMP/college_tost.csv', index=False)

# df.info()
# df.head()

C:\Users\Justin\AppData\Local\Temp\ipykernel_4772\1263327529.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_4772\1263327529.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_4772\1263327529.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_4772\1263327529.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
C:\Users\Justin\AppData\Local\Te